In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 데이터 불러오기
df_c = pd.read_csv("./datasets/raw/연결개별10대산업분류.csv", encoding='cp949')
df_f = pd.read_csv("./datasets/전처리후/재무완성.csv",index_col=0)
df_g = pd.read_csv("./datasets/raw/연결개별G-임금-pre.csv", index_col=0)

In [3]:
# 중복되는 값 삭제
df_c.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
df_c.reset_index(drop=True, inplace=True)

df_g.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
df_g.reset_index(drop=True, inplace=True)


In [4]:
## 회사수와 거래소코드수 차이가나는것 확인
print("회사수 : ", len(df_c["회사명"].unique()))
print("거래소코드수 : ", len(df_c["거래소코드"].unique()))

회사수 :  2605
거래소코드수 :  2634


In [5]:
## 회사수와 거래소코드수 차이가나는것 확인
print("회사수 : ", len(df_g["회사명"].unique()))
print("거래소코드수 : ", len(df_g["거래소코드"].unique()))

회사수 :  2509
거래소코드수 :  2537


In [6]:
# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (df_c[df_c["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(df_c["회사명"].unique())
dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = df_c[df_c["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
df_c = pd.concat(dfs)
df_c.reset_index(drop=True, inplace=True)


In [7]:
# 회사명이 2개인 거래소코드 존재 확인 - 엄슴
list_overlap_code = list()
for i in list(df_c["거래소코드"].unique()):
    if len(df_c[df_c["거래소코드"]==i]["회사명"].unique()) > 1:
        list_overlap_code.append(i)
list_overlap_code

[]

In [8]:
## 기업명 변경
df_c[df_c['거래소코드']=='036420']
df_c.loc[df_c['회사명']=='(주)제이콘텐트리',['회사명']] = '(주)콘텐트리중앙'

In [9]:
# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (df_g[df_g["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(df_g["회사명"].unique())
dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_goncat = df_g[df_g["회사명"]==i]
    df_goncat = df_goncat[df_goncat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_goncat)
df_g = pd.concat(dfs)
df_g.reset_index(drop=True, inplace=True)


In [10]:
# 회사명이 2개인 거래소코드 존재 확인 - 엄슴
list_overlap_code = list()
for i in list(df_g["거래소코드"].unique()):
    if len(df_g[df_g["거래소코드"]==i]["회사명"].unique()) > 1:
        list_overlap_code.append(i)
list_overlap_code

[36420]

In [11]:
## 기업명 변경
df_g[df_g['거래소코드']=='036420']
df_g.loc[df_g['회사명']=='(주)제이콘텐트리',['회사명']] = '(주)콘텐트리중앙'

In [12]:
# 회계년도 년도 추출 후 int화
df_c["회계년도"] = df_c["회계년도"].astype("str").str[:4]
df_c["회계년도"] = df_c["회계년도"].astype("int64")

In [13]:
# 회계년도 년도 추출 후 int화
df_g["회계년도"] = df_g["회계년도"].astype("str").str[:4]
df_g["회계년도"] = df_g["회계년도"].astype("int64")

In [14]:
df_cg= pd.merge(df_c,df_g, on=["회사명", "거래소코드", "회계년도"],how="inner")

In [15]:
# 데이터 병합
df_m=pd.merge(df_f,df_cg,  on=["회사명", "거래소코드", "회계년도"], how="inner")

In [16]:
# 1. 종목코드를 6자리로 통일. ex) 990 -> 000990
def padding(x):
    x = str(x)
    return x.rjust(6, "0")

In [17]:
# 거래소 코드 패딩
df_m["거래소코드"] = df_m["거래소코드"].map(padding)

In [18]:
# 결측치 삭제
df_m.dropna(axis=0, inplace=True)

In [ ]:
df.to_csv('산업분류_G임금.csv',encoding='utf-8-sig')